# Discrete sampling
In the last problem of the first session we sampled with given probabilities using prefix sums and binary search. The sampling code is the bottleneck of the whole process, its running times is 3 times higher than the code for uniform probabilities, and the difference would only increase for larger number of outcomes. In the next two problems we discuss two simple, but powerful techniques one can use to sample in time $O(1)$.

**Problem 2a.** Consider the problem of sampling with known probabilities $p_1,\ldots,p_d$. Suppose that you have a black-box that samples with probabilities $q_1,\ldots,q_d$ that are close to $p_1,\ldots,p_d$, say
$$ \forall_{i=1,\ldots,n} p_i \le (1+\varepsilon)q_i.$$

* How can you use this black-box to sample with probabilities $p_1,\ldots,p_d$? It is expected, that the running time of the algorithm would be non-deterministic.
* Prove that your algorithm is correct.
* Implement the algorithm and use it to give a faster implementation for **Problem 1c**.

**Problem 2b.** One of the reasons this implementation is not significantly faster than the one in **Problem 1c** , apart from $d$ being rather small, is that we are using Python's interpreter a bit too much, and Python is slow. One way around this is usually to use a library function - **searchsorted** is much faster than an equivalent code implemented in pure Python. But even if the functionality you need is not implemented in a lower level language as
a library function, there is still hope. You can try to implement it using optimized array algebra, for example using **numpy**. In this problem, your task is to rewrite the previous algorithm, so that the amount of *looping* is reduced to a minimum necessary. In particular, you should create a *vectorized* version of random dates generation (in bulk), while the actual search for duplicates remains a loop with a **set**. Here are some useful tips:
   * You can perform arithmetic, comparisons, etc. on **numpy** arrays.
   * You can generate whole **numpy** arrays of random numbers at once.
   * You can even perform parallel look-up like in the example below.

In [1]:
X = np.array([10,3,7])
I = np.array([1,1,2,2])
print(X[I])
X = np.array([[1,2],[3,4]])
I = np.array([0,0,1])
J = np.array([1,0,1])
print(X[I,J])

NameError: name 'np' is not defined

**Problem 2c (Squaring the histogram).** In this problem, we again want to sample with known probabilities $p_1,\ldots,p_n$, but this time we make no assumptions on $p_i$. Consider the following algorithm:
   * Let $V$ be the mean of $p_i$, i.e. $V=\frac{1}{n}$.
   * Create $n$ buckets, each with volume $V$, put each $p_i$ into a separate bucket.
   * Until there exists a bucket $A$ that is not full, find a bucket $B$ that overflows, and trasfer probability from $B$ to $A$ until $A$ is exactly full

Show that:
   * This algorithm always ends.
   * When it ends, each bucket contains pieces of exactly two $p_i$'s.

How to use the result of this algorithm to sample with probabilities $p_i$. Argue that your algorithm is correct and implement it. The sampling part should be *vectorized*. Use this algorithm to sample birthdates again, and test its efficiency.

Operation of transferring probability from one bucket to another doesn't change mean ($V = 1/n$).

In each step we have two options:

In [1]:
import numpy as np

p = np.linspace(0, 1, num=5)

def squaring_histogram(p):
    B = np.array(p, dtype='f') # Bucket
    N = B.size

    mean = np.mean(B)
    
    Donor = np.arange(N)
    Probs = np.empty(N)
    Probs.fill(mean)
    
    # original  p_i < Probs[i] | donor|Mean 
    # *************************|******|Mean

    while np.count_nonzero(B < mean):

        i = np.where(B < mean)[0][0]
        j = np.where(B > mean)[0][0]

        Donor[i] = j
        Probs[i] = B[i]
        
        B[j] = B[j] - (mean - B[i])
        B[i] = mean
        
    return Donor, Probs

print squaring_histogram(p)

(array([3, 4, 2, 4, 4]), array([ 0.  ,  0.25,  0.5 ,  0.25,  0.5 ]))


In [35]:
import pandas as pd

data = pd.read_csv('us_births_69_88.csv')
births = np.array(data['births'])

n = births.size
mean = np.mean(births)
K = 400

sample_days = np.random.randint(0, n-1, K)
sample_births = np.random.random_sample(K) * mean

Donor, Probs = squaring_histogram(births)

# Brutal for
days = []
for i in range(K):
    day = sample_days[i]
    day_prob = sample_births[i]
    
    if day_prob < Probs[day]:
        days.append(day)
    else:
        days.append(Donor[day])

# Vectorizing to get the same result as in for above
use_original_value = sample_births < Probs[sample_days]
use_donor_value = ~use_original_value
sample_days[use_donor_value] = Donor[sample_days[use_donor_value]]

# Trying to figure out where repetition took place
_, indx = np.unique(sample_days, return_index=True)
# print(_, indx)
indx.sort()
indx
# indxs = np.arange(K)
# indxs[indx]


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  41,  42,  43,  44,  45,  46,  47,  48,  50,  52,  53,  54,
        56,  57,  58,  59,  60,  61,  62,  64,  66,  69,  70,  71,  72,
        73,  74,  75,  77,  78,  80,  81,  82,  83,  85,  87,  89,  90,
        91,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 126, 127, 128, 129, 130, 132, 133, 135, 138, 139, 140,
       142, 145, 146, 148, 150, 151, 152, 155, 156, 158, 160, 161, 167,
       168, 169, 170, 171, 172, 174, 177, 178, 179, 180, 181, 185, 186,
       187, 190, 191, 192, 193, 195, 197, 198, 200, 201, 202, 203, 204,
       211, 212, 213, 215, 216, 217, 219, 221, 222, 223, 224, 225, 226,
       227, 230, 234, 236, 242, 243, 245, 249, 250, 252, 253, 25

**Problem 2d.** Show that the frequency histogram for empirical birthday frequencies can actually be computed exactly, and implement your idea. To this end, design a recurence relation using conditional probabilities and use dynamic programming.